In [1]:
from xml.etree import ElementTree
from xml.dom import minidom
import glob, os
from xml.etree.ElementTree import Element, SubElement, Comment
import subprocess as sp
import pandas as pd
from datetime import datetime

In [2]:
def prettify(elem):
    """Return a pretty-printed XML string for the Element.
    """
    rough_string = ElementTree.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [3]:
def writeOGRVRT(basename, fullpath):
    top = Element('OGRVRTDataSource')
    child = SubElement(top, 'OGRVRTLayer')
    child.set('name', basename)
    sub_child_1 = SubElement(child, 'SrcDataSource')
    sub_child_1.text = fullpath
    sub_child_2 = SubElement(child, 'GeometryType')
    sub_child_2.text = 'wkbPoint'
    sub_child_3 = SubElement(child, 'LayerSRS')
    sub_child_3.text = 'EPSG:28992'
    sub_child_4 = SubElement(child, 'GeometryField')
    sub_child_4.set('encoding','PointFromColumns')
    sub_child_4.set('x','field_1')
    sub_child_4.set('y','field_2')
    sub_child_4.set('z','field_3')

    return prettify(top)

In [4]:
directory = r'D:\Projects\Pr\3317.20\Toeleveringen\peiling_all'
csv_dir = r'D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\csv_OGRVRT'
vrt_dir = r'D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\vrt_OGRVRT'
shp_dir = r'D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\shp_OGRVRT'
tif_dir = r'D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\tif_OGRVRT'
asc_dir = r'D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\asc_OGRVRT'
ogr2ogr = r'C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe'
gdal_rasterize = r'C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe'
gdal_translate = r'C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe'

In [5]:
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.asc'):
            old_name = os.path.join( os.path.abspath(root), file )
            base, extension = os.path.splitext(file)
            #print (base)

In [6]:
for root, dirs, files in os.walk(directory):
    for file in files:
        if file.endswith('.asc'):
            old_name = os.path.join( os.path.abspath(root), file )
            base, extension = os.path.splitext(file)
            print (base)
            
            #write csv
            csv_out = os.path.join(csv_dir,base+'.csv')
            df = pd.read_csv(old_name, header=None)
            df.to_csv(csv_out, header=False, index=False, sep=';')
            
            # write vrt 
            vrt_out = os.path.join(vrt_dir,base+'.vrt')
            with open(vrt_out, 'w') as the_file:
                the_file.write(writeOGRVRT(base, csv_out))
            
            # convert asc to shp
            shp_out = os.path.join(shp_dir,base+'.shp')
            command = [ogr2ogr, '-f','ESRI Shapefile',shp_out,vrt_out]
            print (sp.list2cmdline(command))
            norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
            norm.communicate()
            
            # convert base date to fews_data 
            # see: https://publicwiki.deltares.nl/display/FEWSDOC/ArcInfoAscii
            dateISO = datetime(int('20'+base[15:17]), int(base[17:19]), int(base[19:21]))
            dateStr = dateISO.strftime("%Y%m%d%M%S")
            
            # convert shp to tif
            tif_out = os.path.join(tif_dir,base+'.tif')
            command = [gdal_rasterize, '-a','field_3','-tr','1.0','1.0','-l',base,shp_out,tif_out]
            print (sp.list2cmdline(command))
            norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
            norm.communicate()  
            
            # get path out
            loc_out = (base[22::]).replace('_','')
            loc_out = loc_out.replace('-','')
            base_par_loc = 'peiling'+loc_out            
            arcinfoascii_out = base_par_loc + '_' + dateStr + '.asc'
            print (arcinfoascii_out)
            
            # convert tif to arcinfoascii [asc]
            asc_out = os.path.join(asc_dir,arcinfoascii_out)
            command = [gdal_translate, '-of','AAIGrid',tif_out,asc_out]
            print (sp.list2cmdline(command))
            norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
            norm.communicate()

SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1
C:\Python35\Lib\site-packages\osgeo//ogr2ogr.exe -f "ESRI Shapefile" D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\shp_OGRVRT\SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1.shp D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\vrt_OGRVRT\SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1.vrt
C:\Python35\Lib\site-packages\osgeo//gdal_rasterize.exe -a field_3 -tr 1.0 1.0 -l SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1 D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\shp_OGRVRT\SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1.shp D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\tif_OGRVRT\SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1.tif
peilingHWKHOKKRMBAJR1x1_201509030000.asc
C:\Python35\Lib\site-packages\osgeo//gdal_translate.exe -of AAIGrid D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\tif_OGRVRT\SUR-5081-ATU-a-150903-HWK_HOK-KR-MB-AJR-1x1.tif D:\Projects\Pr\3317.20\Toeleveringen\peiling_all\asc_OGRVRT\peilingHWKHOKKRMBAJR1x1_201509030000.asc
SUR-5081-AT

ValueError: invalid literal for int() with base 10: '20AJ'